# Terraform Enterprise Install - Stage 1

## Prerequisites

### Set Main Environment Variables

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

printf "${GREEN}# Set MAIN_DIR for future reference.${NC}\n"
export MAIN_DIR=${PWD}
export TF_DIR=config/terraform/terraform-aws-tfe-prereqs/tests/single-region

In [ ]:
DATE=$(date +%Y%m%d) && echo ${DATE}

Customize `friendly_name_prefix`, `AWS_REGION`, and `cidr_block`.

In [ ]:
# TF_DIR=config/terraform/tfe_stage_1
export AWS_REGION=us-west-2 && AWS_DEFAULT_REGION=${AWS_REGION} && region=${AWS_REGION}
friendly_name_prefix=pphan-${DATE}
cidr_block="10.0.0.0/16"
bastion_ip=$(curl -s http://ipv4.icanhazip.com)

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

printf "${GREEN}# Set MAIN_DIR for future reference.${NC}\n"
export MAIN_DIR=${PWD}

Be sure to replace `PREFIX`, `TF_ORG`, and `TF_WS_NAME` with your own values.

In [ ]:
export PREFIX=pphan
export TF_ORG=pphan
export TFC_ORG=$TF_ORG
export TF_WS_NAME=consumer
export TOKEN=$(jq -r '.credentials."app.terraform.io".token' ~/.terraform.d/credentials.tfrc.json); export TFE_TOKEN=$TOKEN

export TF_GIT_DIR="config/tfc-agent"
export TFE_PROVIDER_DIR="tfe-provider"
printf "${GREEN}# Set Terraform Variables.${NC}\n"
printf "\n$MAIN_DIR \n $TF_GIT_DIR \n TFE_PROVIDER_DIR: $TFE_PROVIDER_DIR
 TOKEN: $TOKEN \n TF_ORG: $TFC_ORG
 TF_WS_NAME: $TF_WS_NAME"

### AWS Credentials

Unset any previous AWS credentials in environment variables.

In [ ]:
unset AWS_SESSION_TOKEN AWS_SECRET_ACCESS_KEY AWS_ACCESS_KEY_ID

Set the AWS credentials

In [ ]:
export AWS_ACCESS_KEY_ID=REPLACE_ME
export AWS_SECRET_ACCESS_KEY=REPLACE_ME

For Hashi only

In [ ]:
awscredsenv

Confirm the credentials are in place. Optional link for [envo](https://github.com/im2nguyen/envo).

In [ ]:
envo | grep AWS || env | grep AWS

## Single Region
In this example, we will provision resources in a single primary region.

### Resources Provisioned and Omitted

| Resource | Single Region | Primary Region with S3 CRR | Primary and Secondary Region |
| --- | --- | --- | --- |
| VPC | Yes |
  | Public Subnets | Yes |
  | Private Subnets | Yes |
  | Internet Gateway | Yes |
  | NAT Gateways (for each Public Subnet) | Yes |
  | Route Tables and Routes | Yes |
  | S3 VPC Endpoint | Yes |
| Bastion host | Yes |
| S3 "bootstrap" bucket (Primary) | Yes |
| KMS Key | Yes |
| AWS Secrets Manager secret | Yes |
| AWS CloudWatch Log Group | Yes |
| S3 "bootstrap" bucket **replica** in Secondary (Disaster Recovery) region | NO |
| S3 Cross-Region Replication (CRR) configuration between 'Primary' and 'Replica' buckets | NO |
| S3 "logging" bucket | NO |
| AWS ssh key pair - NEED TO CONFIRM | NO |

| Resource |
| --- |
| VPC
| Public Subnets
| Private Subnets
| Internet Gateway
| NAT Gateways (for each Public Subnet)
| Route Tables and Routes
| S3 VPC Endpoint
| Bastion host
| S3 "bootstrap" bucket (Primary)
| KMS Key
| AWS Secrets Manager secret
| AWS CloudWatch Log Group

## Clone the Repo - Hashi Folks

Create a directory such as `config/terraform/terraform-aws-tfe` into which you want to clone this repository.

In [ ]:
mkdir -p config/terraform/terraform-aws-tfe-prereqs

Clone the repo.

In [ ]:
git clone https://github.com/hashicorp-services/terraform-aws-tfe-prereqs config/terraform/terraform-aws-tfe-prereqs

### unzip Repo - For Non-Hashi Folks

A zip file of the Terraform module `terraform-aws-tfe.zip` is in this repo. You can check with your account team in case there is an updated version.

Extract the archive.

In [ ]:
mkdir -p config/terraform && \
  unzip -qq terraform-aws-tfe-prereqs.zip -d config/terraform/ && \
  ls config/terraform/terraform-aws-tfe-prereqs

## Set Terraform Variables

Confirm that TF_DIR is mapped to the single-region directory that contains the **Stage 1** Terraform code.
	* Run `cd examples/aws/network-public` or `cd examples/aws/network-private` 


In [ ]:
echo $TF_DIR

Create an `auto.tfvars` file.

### Set Variables auto.tfvars

Customize `terraform.auto.tfvars`.
* Set `friendly_name_prefix` to some suitable prefix for your TFE deployment
* Set `aws_region` to your AWS region
* Set `cidr_block` to a valid CIDR block: ex `cidr_block = "10.110.0.0/16"`
* Set `*_password` to your own passwords
* Set `bastion_keypair` to your key. We will create one as a modification to this module.

In [ ]:
tee $TF_DIR/terraform.auto.tfvars <<EOF
#// Primary Region
# --- Common --- #
friendly_name_prefix = "${friendly_name_prefix}"
common_tags = {
  "Environment" = "tfe-prereqs-primary"
  "Tool"        = "Terraform"
  "Owner"       = "YourName"
}

# --- Network --- #
deploy_vpc           = true
vpc_cidr             = "${cidr_block}"
public_subnet_cidrs  = ["10.0.1.0/24", "10.0.2.0/24", "10.0.3.0/24"]
private_subnet_cidrs = ["10.0.255.0/24", "10.0.254.0/24", "10.0.253.0/24"]

# --- Bastion --- #
deploy_bastion             = true
bastion_keypair            = "${friendly_name_prefix}-bastion-key-pair-${AWS_REGION}"
bastion_ingress_cidr_allow = ["${bastion_ip}/32"]

# --- S3 --- #
deploy_bootstrap_bucket   = true
bootstrap_bucket_name     = "${friendly_name_prefix}-tfe-bootstrap-primary-${DATE}1"

# --- KMS --- #
deploy_kms = true

# --- Secrets Manager --- #
deploy_secretsmanager      = true
secretsmanager_secret_name = "${friendly_name_prefix}-tfe-install-secrets"
secretsmanager_secrets = {
  console_password            = "ProtectThisSecretBetter123!"
  enc_password                = "DefinitelyProtectThisOne456$"
  # tfe_initial_admin_password  = "AdvancedTfeBootstrapFunctionality789@"
}

# --- CloudWatch Logs --- #
deploy_log_group         = true
log_group_name           = "vc-hcl-tfe-online"
log_group_retention_days = 30
EOF

## Modifications to main.tf

The example config has the AWS region statically set. We will undo this.

Comment out `region` parameter in AWS provider.

In [ ]:
sed -i '' '/^[^#]/ s/\(^.* region .*$\)/#\ \1/' $TF_DIR/main.tf

## Optional Configuration for SSH Keys

Create items not created by the module eg AWS ssh key. We will upload our ssh key. This is not necessary if you already have an SSH key. Just specify it in the tfvars file above.

In [ ]:
cat > ${TF_DIR}/aws_key_pair.tf <<EOF
resource "aws_key_pair" "tfe" {
  key_name   = "${friendly_name_prefix}-bastion-key-pair-${AWS_REGION}"
  public_key = file("~/.ssh/id_rsa.pub") #var.ssh_public_key
}
output "ssh_key_pair" {
  value = aws_key_pair.tfe.id
}
EOF

## Terraform Init

Initialize **Stage 1** Terraform configuration and download providers.

In [ ]:
terraform -chdir=${TF_DIR} init

## Terraform Plan

* We will format (`fmt`) the configuration and `validate` it.
* If validation passes, then we will do a `terraform plan` for a dry run.

In [ ]:
printf "${GREEN}#--> Reformat your configuration in the standard style${NC}\n" && \
terraform -chdir=${TF_DIR} fmt && \
printf "${GREEN}#--> Check whether the configuration is valid.${NC}\n" && \
terraform -chdir=${TF_DIR} validate && \
printf "${GREEN}#--> Show changes required by the current configuration.${NC}\n" && \
# terraform -chdir=${TF_DIR} plan -input=false

## Terraform Apply - Provision the resources

The apply takes approximate three minutes. Output is sent to `/tmp/tf_stage_1_apply.out`. 

In [ ]:
time terraform -chdir=${TF_DIR} apply -input=false -auto-approve \
  > /tmp/tf_stage_1_apply.out 2>&1

You can monitor progress from a separate terminal with this command.
```shell
tail -f /tmp/tf_stage_1_apply.out
```

Show terraform outputs.

In [ ]:
terraform -chdir=${TF_DIR} output

## Note the outputs

Note the following outputs, which you will need in **Stage 2**.

* `kms__key_id`
* `security_group_id`
* `*_subnet_ids`
* `vpc_id`

Sample Output

```shell
Apply complete! Resources: 44 added, 0 changed, 0 destroyed.

Outputs:

bastion_private_ip = "10.0.1.210"
bastion_public_dns = "ec2-54-190-138-113.us-west-2.compute.amazonaws.com"
bastion_public_ip = "54.190.138.113"
bootstrap_bucket_arn = "arn:aws:s3:::pphan-20211028-tfe-bootstrap-primary-123456789"
bootstrap_bucket_name = "pphan-20211028-tfe-bootstrap-primary-123456789"
kms_key_arn = "arn:aws:kms:us-west-2:<aws_acct_id>:key/249cf3dd-4eff-4041-ae22-3c352bdc0676"
kms_key_id = "249cf3dd-4eff-4041-ae22-3c352bdc0676"
log_bucket_name = ""
log_group_name = "vc-hcl-tfe-online"
private_subnet_ids = [
  "subnet-007c20dac236ecad3",
  "subnet-0c7e5c7d5aaf848c6",
  "subnet-0b6f4a69f36013b82",
]
public_subnet_ids = [
  "subnet-03b8b2c84dfb76e0b",
  "subnet-02168a862dadcd51c",
  "subnet-01728482622f84d85",
]
s3_replication_iam_role_arn = ""
secretsmanager_secret_arn = "arn:aws:secretsmanager:us-west-2:<aws_acct_id>:secret:pphan-20211028-tfe-install-secrets-HoNvCs"
ssh_key_pair = "pphan-20211028-bastion-key-pair-us-west-2"
tfe_ssh_keypair_fingerprint = ""
tfe_ssh_keypair_id = ""
tfe_ssh_keypair_name = ""
vpc_id = "vpc-03be0c1b68d9377b1"
```

## Copy license file to S3 Bootstrap bucket

Add your TFE license file to your TFE source bucket that was created.

The easy method is to use the AWS S3 console.
* Go to your bucket and upload the file.

Alternatively, you can do it from the CLI.
1. Place your TFE license file in `${TF_DIR}` folder.
1. Copy it to your S3 bootstrap bucket.

In [ ]:
export BOOTSTRAP_BUCKET_NAME_PRIMARY=$(terraform -chdir=${TF_DIR} output -raw bootstrap_bucket_name_primary) && echo $BOOTSTRAP_BUCKET_NAME_PRIMARY

In [ ]:
aws s3 cp ${TF_DIR}/tfe-license.rli s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}/

3. Confirm file was uploaded to AWS S3.

In [ ]:
aws s3 ls s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}

* Name the various objects in your TFE source bucket to match the values given in the `tfvars` file.
    * Take into account the version of your airgap bundle.
* Avoid the use of spaces in the names of the TFE license and installation files.
* If doing an airgapped installation, add your airgap bundle and `replicated.tar.gz` to the PTFE source bucket too.

```shell
aws s3 cp replicated.tar.gz s3://pphan-tfe-source-bucket
```

## Clean Up

### Delete S3

Delete license file from S3 bucket.

In [ ]:
aws s3 rm s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}/tfe-license.rli

In [ ]:
aws s3 ls s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}

Get version id of license file.

In [ ]:
LICENSE_VERSION_ID=$(aws s3api list-object-versions --bucket ${BOOTSTRAP_BUCKET_NAME_PRIMARY} --prefix tfe-license.rli | jq -r .Versions[].VersionId) && echo ${LICENSE_VERSION_ID}

Permanently delete license file.

In [ ]:
aws s3api delete-object --bucket ${BOOTSTRAP_BUCKET_NAME_PRIMARY} --key tfe-license.rli --version-id ${LICENSE_VERSION_ID}

### Destroy - Stage 1 Resources

If no longer needed, you can destroy your provisioned resources. This takes 3 minutes.

In [ ]:
terraform -chdir=${TF_DIR} destroy -input=false -auto-approve \
  > /tmp/tf_stage_1_destroy.out 2>&1

### Delete AWS Secret Immediately

When secrets are marked for deletion, there will be a default recovery window of 14 days. To immediately delete the secret, perform these steps.

* Go to AWS Secrets Manager to see your secrets. 
* On the Secrets page, choose Preferences (Gear icon).
* In the Preferences dialog box, select **Show disabled secrets**, and then choose **Save**.

In [ ]:
open https://${AWS_REGION}.console.aws.amazon.com/secretsmanager/home?region=${AWS_REGION}\#\!/listSecrets

Force immediate deletion, so that you can reuse the same name.

In [ ]:
aws secretsmanager delete-secret --secret-id ${friendly_name_prefix}-tfe-install-secrets --force-delete-without-recovery

In [ ]:
aws secretsmanager list-secrets

Note the `DeletionDate` shows the current date and time instead of a future date.

### Delete file artifacts

In [ ]:
ls -l $MAIN_DIR/config/terraform/terraform-aws-tfe-prereqs

In [ ]:
cd $MAIN_DIR && rm -rf config/terraform/terraform-aws-tfe-prereqs

## Experimental

### Create configuration

Don't use this for now.

In [ ]:
cat > config/terraform/tfe_stage_1/main.tf <<EOF
#// Versions
terraform {
  required_version = "~> 0.14.7"
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 3.63.0"
    }
    template = {
      source  = "hashicorp/template"
      version = "~> 2.2.0"
    }
  }
}

#// Provider
provider "aws" {
  region = "${region}"
}
EOF

In [ ]:
tee config/terraform/tfe_stage_1/main.tf <<EOF
#// Primary Region
module "tfe-prereqs-primary" {
  source = "github.com/hashicorp/is-terraform-aws-tfe-prereqs"

  # --- Common --- #
  friendly_name_prefix = "${friendly_name_prefix}"
  common_tags = {
    "Environment" = "tfe-prereqs-primary"
    "Tool"        = "Terraform"
    "Owner"       = "Clegane"
  }

  # --- Network --- #
  deploy_vpc           = true
  vpc_cidr             = "${cidr_block}"
  public_subnet_cidrs  = ["10.0.1.0/24", "10.0.2.0/24", "10.0.3.0/24"]
  private_subnet_cidrs = ["10.0.255.0/24", "10.0.254.0/24", "10.0.253.0/24"]

  # --- Bastion --- #
  deploy_bastion             = true
  bastion_keypair            = aws_key_pair.tfe.key_name #"${friendly_name_prefix}-bastion-key-pair-us-east-1"
  bastion_ingress_cidr_allow = ["${bastion_ip}/32"]

  # --- S3 --- #
  deploy_bootstrap_bucket   = true
  bootstrap_bucket_name     = "${friendly_name_prefix}-tfe-bootstrap-primary-123456789"

  # --- KMS --- #
  deploy_kms = true

  # --- Secrets Manager --- #
  deploy_secretsmanager      = true
  secretsmanager_secret_name = "${friendly_name_prefix}-tfe-install-secrets"
  secretsmanager_secrets = {
    console_password            = "ProtectThisSecretBetter123!"
    enc_password                = "DefinitelyProtectThisOne456$"
    tfe_initial_admin_password  = "AdvancedTfeBootstrapFunctionality789@"
  }
}

output "vpc" {
  value = module.tfe-prereqs-primary.vpc_id
}

output "public_subnet_ids" {
  value = module.tfe-prereqs-primary.public_subnet_ids
}

output "private_subnet_ids" {
  value = module.tfe-prereqs-primary.private_subnet_ids
}

output "bootstrap_bucket_name_primary" {
  value = module.tfe-prereqs-primary.bootstrap_bucket_name
}

output "bastion_public_ip" {
  value = module.tfe-prereqs-primary.bastion_public_ip
}

output "kms_key_arn" {
  value = module.tfe-prereqs-primary.kms_key_arn
}

output "secretsmanager_secret_metadata_arn" {
  value = module.tfe-prereqs-primary.secretsmanager_secret_arn
}
EOF